In [1]:
import pandas as pd
import numpy as np

In [2]:
soubor = pd.read_csv('clean_data_years_filter.csv',  encoding='utf-8')
soubor.head(2)

,Období,Typ odběratele,ATC7,Kód SÚKL,Název přípravku,Doplněk názvu,Držitel registračního rozhodnutí,Země,Typ pohybu,Počet balení/M,Cena za balení bez obch. přirážky a DPH,Celkem finance za všechna balení bez obch. přirážky a DPH,Počet definovaných denních dávek/balení,Počet denních definovaných dávek/balení celkem,Způsob výdeje
0,2011-01-01,LEKAR,M01AE01,125526.0,APO-IBUPROFEN 400 MG,POR TBL FLM 100X400MG,ATX,NL,D,10.0,82.83,828.3,33.3333,333.0,F
1,2011-01-01,LEKARNA,M01AE01,125526.0,APO-IBUPROFEN 400 MG,POR TBL FLM 100X400MG,ATX,NL,D,71070.0,82.83,5886728.1,33.3333,2368998.0,F


###### "clean_data_years_filter.csv" obsahuje data, kde:

###### "Typ odběratele":
'LEKARNA', 'VETERINARNI_LEKAR', 'PRODEJCE_VLP','HYGIENICKA_STANICE', 'ZDRAVOTNICKE_ZARIZENI', 'LEKAR', 'OSOBA_POSKYTUJICI_ZDRAVOTNI_PECI'

###### Abychom nic nepokazily, vytvorime novou tabulku "soubor1" a přejmenujeme poslední sloupec

In [3]:
soubor1 = soubor.loc[:,['Období', 'ATC7', 'Počet denních definovaných dávek/balení celkem']]

In [4]:
soubor1 = soubor1.rename(index=str, columns={'Počet denních definovaných dávek/balení celkem': 'DDD_total'})
soubor1.head()

,Období,ATC7,DDD_total
0,2011-01-01,M01AE01,333.0
1,2011-01-01,M01AE01,2368998.0
2,2011-01-01,M01AE01,-45533.0
3,2011-01-01,M01AE01,600.0
4,2011-01-01,M01AE01,20.0


### Zkusime spojit jednotlive ATC do vyssi urovni molekuly. 

Proto napred socteme vsechny ATC, nazev kterych se zacina s N02:

Jsou:  'N02B', 'N02BA', 'N02BA01','N02BA02', 'N02BA03','N02BA04', 'N02BA05', 'N02BA06','N02BA07', 'N02BA08', 'N02BA09', 'N02BA10', 'N02BA11', 'N02BA12', 'N02BA14', 'N02BA15', 'N02BA16', 'N02BA51', 'N02BA55', 'N02BA57', 'N02BA59', 'N02BA65', 'N02BA71', 'N02BA75', 'N02BA77', 'N02BA79', 'N02BE', 'N02BE01', 'N02BE03', 'N02BE04', 'N02BE05', 'N02BE51', 'N02BE53', 'N02BE54', 'N02BE71', 'N02BE73', 'N02BE74'








### Jako další krok bylo třeba seskupit všechny léky obsahující stejnou molekulu ATC. 
Stejné ATC totiž může být základem pro různé léky. Použily jsme metodu groupby, pomocí které jsme vytvořily součet napříč jednotlivými ATC za každý měsíc. Protože k tomuto výpočtu z původních patnácti sloupců v tabulce nám vystačí tři, jsme do syntaxe zapsaly jen “Období”, “ATC” a “DDD_total”. 

In [5]:
soubor1[soubor1['ATC7'].str.startswith('N02')].shape

(19168, 3)

In [6]:
leky_N02 = soubor1[soubor1['ATC7'].str.startswith('N02')]
leky_N02.tail(3)

,Období,ATC7,DDD_total
87352,2018-03-01,N02BE51,NaN
87353,2018-03-01,N02BE51,NaN
87354,2018-03-01,N02BE51,NaN


In [7]:
leky_N02 = leky_N02.groupby(['Období']).agg({'Období':'first', 'DDD_total':'sum'})
leky_N02.tail()

,Období,DDD_total
Období,,
2017-11-01,2017-11-01,3.550867e+06
2017-12-01,2017-12-01,2.783926e+06
2018-01-01,2018-01-01,4.502959e+06
2018-02-01,2018-02-01,5.323285e+06
2018-03-01,2018-03-01,4.917765e+06


In [8]:
leky_N02.shape

(87, 2)

In [9]:
leky_N02.columns

Index(['Období', 'DDD_total'], dtype='object')

In [10]:
# Exportujeme
leky_N02.to_csv('leky/leky_N02.csv', encoding = 'utf-8')

In [11]:
leky_N02['Období'] = pd.to_datetime(leky_N02['Období'])

In [12]:
leky_N02['year'] = leky_N02['Období'].dt.year.astype(np.int)

In [13]:
leky_N02['month'] = leky_N02['Období'].dt.month.astype(np.int)

In [14]:
leky_N02['quater'] = leky_N02['Období'].dt.quarter.astype(np.int)
#DatetimeIndex.quarter

#### Pro statistický model jsme potřebovaly vytvořit “trend”. Náš nejstarší rok je označen nulou a každý další je o jednotku vyšší.

In [15]:
leky_N02['trend'] = leky_N02['year']-2011

In [16]:
leky_N02.tail(3)

,Období,DDD_total,year,month,quater,trend
Období,,,,,,
2018-01-01,2018-01-01,4.502959e+06,2018,1,1,7
2018-02-01,2018-02-01,5.323285e+06,2018,2,1,7
2018-03-01,2018-03-01,4.917765e+06,2018,3,1,7


### Statistický model
Pro použití statistického modelu jsme potřebovaly vytvořit další sloupce pro rok, měsíc a kvartál pomocí metody knihovny pandas “.to_datetime”.

Program vnímá čísla měsíců (1–12) a kvartálů (1–4) jako hodnoty, což se nedá využít pro náš stats model. Aby program nevnímal číslo měsíců a kvartálů jako hodnotu, je třeba použít binární systém jedniček a nul.

### Proto použijeme metodu “.get_dummies” na sloupce “month” a “quater”.

In [17]:
Q1 = pd.get_dummies(leky_N02['quater'], prefix='quater')
Q1.head(3)

,quater_1,quater_2,quater_3,quater_4
Období,,,,
2011-01-01,1,0,0,0
2011-02-01,1,0,0,0
2011-03-01,1,0,0,0


In [18]:
M1 = pd.get_dummies(leky_N02['month'], prefix='month')
M1.head()

,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
Období,,,,,,,,,,,,
2011-01-01,1,0,0,0,0,0,0,0,0,0,0,0
2011-02-01,0,1,0,0,0,0,0,0,0,0,0,0
2011-03-01,0,0,1,0,0,0,0,0,0,0,0,0
2011-04-01,0,0,0,1,0,0,0,0,0,0,0,0
2011-05-01,0,0,0,0,1,0,0,0,0,0,0,0


### Ted to spojime do jednoho dataframu "model_df"
https://pandas.pydata.org/pandas-docs/stable/merging.html

In [19]:
model_N02 = pd.concat([leky_N02, M1, Q1], axis=1, join_axes=[leky_N02.index])
model_N02.head(3)

,Období,DDD_total,year,month,quater,trend,month_1,month_2,month_3,month_4,...,month_7,month_8,month_9,month_10,month_11,month_12,quater_1,quater_2,quater_3,quater_4
Období,,,,,,,,,,,,,,,,,,,,,
2011-01-01,2011-01-01,5386347.0,2011,1,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2011-02-01,2011-02-01,4883015.0,2011,2,1,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
2011-03-01,2011-03-01,4549898.0,2011,3,1,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0


In [20]:
model_N02.info()

<class 'pandas.core.frame.DataFrame'>
Index: 87 entries, 2011-01-01 to 2018-03-01
Data columns (total 22 columns):
Období       87 non-null datetime64[ns]
DDD_total    87 non-null float64
year         87 non-null int32
month        87 non-null int32
quater       87 non-null int32
trend        87 non-null int32
month_1      87 non-null uint8
month_2      87 non-null uint8
month_3      87 non-null uint8
month_4      87 non-null uint8
month_5      87 non-null uint8
month_6      87 non-null uint8
month_7      87 non-null uint8
month_8      87 non-null uint8
month_9      87 non-null uint8
month_10     87 non-null uint8
month_11     87 non-null uint8
month_12     87 non-null uint8
quater_1     87 non-null uint8
quater_2     87 non-null uint8
quater_3     87 non-null uint8
quater_4     87 non-null uint8
dtypes: datetime64[ns](1), float64(1), int32(4), uint8(16)
memory usage: 4.8+ KB


In [21]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

C:\Users\Katerina\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Pouzijeme agregacni funkce a udelame soucet za kazdy mesic:

In [22]:
resultsN02 = smf.ols('DDD_total ~ trend + quater_1 + quater_2 + quater_3', data=model_N02).fit()
print(resultsN02.summary())

                            OLS Regression Results                            
Dep. Variable:              DDD_total   R-squared:                       0.301
Model:                            OLS   Adj. R-squared:                  0.267
Method:                 Least Squares   F-statistic:                     8.837
Date:                Sun, 20 May 2018   Prob (F-statistic):           5.54e-06
Time:                        19:21:03   Log-Likelihood:                -1311.3
No. Observations:                  87   AIC:                             2633.
Df Residuals:                      82   BIC:                             2645.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   3.811e+06   2.34e+05     16.278      0.0

In [23]:
results_N02 = smf.ols('DDD_total ~ trend + month_1 + month_2 + month_3 + month_4 + month_5 + month_6 + month_7 + month_8 + month_9 + month_10 + month_11', data=model_N02).fit()
print(results_N02.summary())

                            OLS Regression Results                            
Dep. Variable:              DDD_total   R-squared:                       0.619
Model:                            OLS   Adj. R-squared:                  0.558
Method:                 Least Squares   F-statistic:                     10.04
Date:                Sun, 20 May 2018   Prob (F-statistic):           2.51e-11
Time:                        19:21:03   Log-Likelihood:                -1284.9
No. Observations:                  87   AIC:                             2596.
Df Residuals:                      74   BIC:                             2628.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   3.732e+06   2.78e+05     13.432      0.0